In [2]:
import numpy as np
from quaos.symplectic import PauliSum, PauliString, Pauli, Xnd, Ynd, Znd, Id, string_to_symplectic, symplectic_to_string
from quaos.gates import GateOperation, Circuit, Hadamard as H, SUM as CX, PHASE as S
from quaos.hamiltonian import *
from quaos.core.prime_Functions_Andrew import int_to_bases
from collections import defaultdict
import sympy as sym
from sympy.physics.quantum import TensorProduct,Operator

In [3]:
def group_indices(lst):
    """
    Groups indices of the same value in a list into sublists.
    For example, if the input list is [1, 2, 1, 3, 2], the output will be [[0, 2], [1, 4], [3]].
    """
    index_dict = defaultdict(list)
    for idx, value in enumerate(lst):
        index_dict[value].append(idx)
    
    return [indices for indices in index_dict.values()]


def Hadamard_Symmetric_PauliSum(n_paulis,n_qubits,n_sym_q):
    # create coefficients
    c_int_bands = np.sort(np.random.randint(n_paulis,size=n_paulis))
    c_bands = group_indices(c_int_bands)

    coefficients = np.zeros(n_paulis)
    sym_bands = []
    for i,b in enumerate(c_bands):
        coefficients[b] = np.random.normal(0, 1)
        if len(b) != 1:
            sym_bands.append(b)

    #print(sym_bands)
    n_extra_bands = np.floor(np.sum([len(b) for b in sym_bands])/2 - n_sym_q)
    pauli_strings = ['' for i in range(n_paulis)]

    all_x = []
    all_z = []
    for i in range(n_sym_q):
        x_pauli = []
        z_pauli = []
        if len(sym_bands) >= 1:
            b_ind = np.random.randint(len(sym_bands))
            b = sym_bands[b_ind]
            x_ind = np.random.randint(len(b))
            x_pauli.append(b[x_ind])
            b.pop(x_ind)
            z_ind = np.random.randint(len(b))
            z_pauli.append(b[z_ind])
            b.pop(z_ind)
            if len(b) < 2:
                sym_bands.pop(b_ind)
            else:
                sym_bands[b_ind] = b

            # randomly adding extra x and zs if possible
            if n_extra_bands > 0 and len(sym_bands) >= 1:
                extras = np.random.randint(n_extra_bands)
                n_extra_bands -= extras
                for j in range(extras):
                    b_ind = np.random.randint(len(sym_bands))
                    b = sym_bands[b_ind]
                    x_ind = np.random.randint(len(b))
                    x_pauli.append(b[x_ind])
                    b.pop(x_ind)
                    z_ind = np.random.randint(len(b))
                    z_pauli.append(b[z_ind])
                    b.pop(z_ind)
                    if len(b) < 2:
                        sym_bands.pop(b_ind)
                    else:
                        sym_bands[b_ind] = b
        print(coefficients[x_pauli])
        for j in range(n_paulis):
            if j in x_pauli:
                pauli_strings[j] += 'x1z0 '
            elif j in z_pauli:
                pauli_strings[j] += 'x0z1 '
            else:
                pauli_strings[j] += 'x0z0 '
        all_x += x_pauli
        all_z += z_pauli
    print(all_x,all_z)
    non_sym_paulis = [i for i in range(n_paulis) if not i in all_x and not i in all_z]
    q_dims = [2 for i in range(2*(n_qubits-n_sym_q))]
    available_paulis = list(np.arange(int(np.prod(q_dims))))
    for i,x in enumerate(all_x):
        pauli_index = np.random.choice(available_paulis)
        available_paulis.remove(pauli_index)
        exponents = int_to_bases(pauli_index, q_dims)
        for j in range(n_qubits-n_sym_q):
            r, s = int(exponents[2*j]), int(exponents[2*j+1])
            pauli_strings[x] += f"x{r}z{s} "
            pauli_strings[all_z[i]] += f"x{r}z{s} "

        pauli_strings[x].strip()
        pauli_strings[all_z[i]].strip()

    for i in non_sym_paulis:
        pauli_index = np.random.choice(available_paulis)
        available_paulis.remove(pauli_index)
        exponents = int_to_bases(pauli_index, q_dims)
        for j in range(n_qubits-n_sym_q):
            r, s = int(exponents[2*j]), int(exponents[2*j+1])
            pauli_strings[i] += f"x{r}z{s} "
        pauli_strings[i].strip()

    P = PauliSum(pauli_strings, weights=coefficients ,dimensions=[2 for i in range(n_qubits)], phases=None,standardise=False)
    #print(P)

    # construct random Clifford circuit
    C = Circuit(dimensions=[2 for i in range(n_qubits)])
    gate_list = [H,S,CX]
    gg = []
    for i in range(100):
        g_i = np.random.randint(3)
        if g_i == 2:
            aa = list(random.sample(range(n_qubits), 2))
            gg += [gate_list[g_i](aa[0],aa[1],2)]
            #print(aa)
        else:
            aa = list(random.sample(range(n_qubits), 1))
            gg += [gate_list[g_i](aa[0],2)]
        
    C.add_gate(gg)
    P = C.act(P)

    phases = P.phases
    cc = P.weights
    ss = P.pauli_strings
    dims = P.dimensions

    cc *= np.array([-1]*n_paulis)**phases
    P = PauliSum(ss, weights=cc ,dimensions=dims, phases=None,standardise=False)

    # qubits shuffle qubits (Fisher Yates Shuffle)
    '''
    gg = []
    order = np.arange(n_qubits)
    for i in np.arange(n_qubits)[::-1]:
        j = np.random.randint(i+1)
        gg += [GateOperation('SWAP',(i,j),['x1z0 x0z0 -> x0z0 x1z0', 'x0z1 x0z0 -> x0z0 x0z1', 'x0z0 x1z0 -> x1z0 x0z0', 'x0z0 x0z1 -> x0z1 x0z0'],2)]
        a = int(order[i])
        b = int(order[j])
        order[i] = b
        order[j] = a
        
    for g in gg:
        P = g.act(P)
    sym_qubit_ind = []
    for i in range(n_qubits):
        if order[i] in np.arange(n_sym_q):
            sym_qubit_ind.append(i)
    '''
    #print('Symmetric qubits: ',sym_qubit_ind)
    #print(P)
    return(P,C)

def add_s2(qubit):
    C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[CX(0,qubit,2),H(qubit,2),CX(qubit,0,2),S(qubit,2),H(qubit,2)])
    return(C)

def add_r2(qubit):
    C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[S(0,2),CX(qubit,0,2),S(0,2)])
    return(C)

def add_r2s2(qubit):
    C = Circuit(dimensions=[2 for i in range(P.n_qudits())],gates=[S(qubit,2),CX(0,qubit,2),H(qubit,2),CX(qubit,0,2),S(qubit,2),H(qubit,2)])
    return(C)

def number_of_I(P,Pauli_index,qubits):
    """
    Returns the number of I's in the Pauli string at Pauli_index for qubits.
    """

    return np.sum([1 for i in qubits if P.x_exp[Pauli_index,i] == 0 and P.z_exp[Pauli_index,i] == 0])

def flatten_list(nested_list):
    """
    Flattens a list of lists into a single list with all the elements.
    
    Args:
        nested_list (list of lists): The input list of lists.
    
    Returns:
        list: A single flattened list containing all elements.
    """
    return [item for sublist in nested_list for item in sublist]

def find_circuit(start_pauli,goal_pauli,iterations,compare_phases = False):
    n_qudits = start_pauli.n_qudits()
    SUMs = [CX(i, j, 2) for i in range(n_qudits) for j in range(n_qudits) if i != j]
    Ss = [S(i, 2) for i in range(n_qudits)]
    Hs = [H(i, 2) for i in range(n_qudits)]
    all_gates = SUMs + Ss + Hs

    goal_circuits = []
    circuits = [Circuit(dimensions=[2 for i in range(n_qudits)])]
    intermediate_paulis = [start_pauli.copy()]
    for i in range(iterations):
        print(i,len(intermediate_paulis),len(goal_circuits))
        intermediate_paulis_old = intermediate_paulis.copy()
        for i,p in enumerate(intermediate_paulis_old):
            for g in all_gates:
                P_temp = g.act(p)
                if not compare_phases:  
                    P_temp.phases = [0] 
                C_temp = Circuit(dimensions=[2 for i in range(n_qudits)])
                for g2 in circuits[i].gates:
                    C_temp.add_gate(g2)
                C_temp.add_gate(g)
                if P_temp not in intermediate_paulis:
                    intermediate_paulis.append(P_temp)
                    circuits.append(C_temp)
                if P_temp == goal_pauli:
                    goal_circuits.append(C_temp)
                    #print("Found goal circuit") 
    return(goal_circuits)

def modulo_2(expr):
    """
    Takes a SymPy expression and reduces its coefficients modulo 2.
    """
    # Expand the expression to handle all terms
    expr = expr.expand()
    
    # Iterate through the terms and apply modulo 2 to coefficients
    terms = expr.as_ordered_terms()
    mod_expr = sum(sym.Mod(term.as_coeff_Mul()[0], 2) * term.as_coeff_Mul()[1] for term in terms)
    
    return mod_expr

def reduce_exponents(expr):
    """
    Reduces all exponents in a SymPy expression to zero, assuming symbols are binary (0 or 1).
    
    Args:
        expr (sympy.Expr): The input SymPy expression.
    
    Returns:
        sympy.Expr: The modified expression with all exponents set to zero.
    """
    expr = sym.expand(expr)  # Expand the expression to handle all terms
    return expr.replace(lambda x: x.is_Pow, lambda x: x.base)

def symplectic_effect(circuit):
    n_qudits = len(circuit.dimensions)
    #print(n_qudits)
    r_now = list(sym.symbols([f'r{i}' for i in range(1, n_qudits+1)]))
    omega = sym.symbols('omega')
    #print(len(r_now))
    s_now = list(sym.symbols([f's{i}' for i in range(1, n_qudits+1)]))
    r_next = [r_now[i] for i in range(n_qudits)]
    s_next = [s_now[i] for i in range(n_qudits)]

    #r1_start, r2_start, s1_start, s2_start = sym.symbols('r1 r2 s1 s2')
    X = Operator('X')
    Z = Operator('Z')

    phase = 0
    gates = circuit.gates
    qubits = circuit.indexes
    for i,g in enumerate(gates):
        if g.name == 'SUM':
            r_next[qubits[i][1]] = r_now[qubits[i][0]] + r_now[qubits[i][1]]
            s_next[qubits[i][0]] = s_now[qubits[i][0]] + s_now[qubits[i][1]]
        elif g.name == 'H' or g.name == 'HADAMARD':
            r_next[qubits[i][0]] = s_now[qubits[i][0]]
            s_next[qubits[i][0]] = r_now[qubits[i][0]]
            phase += s_now[qubits[i][0]] * r_now[qubits[i][0]]
        elif g.name == 'S' or g.name == 'PHASE':
            s_next[qubits[i][0]] = s_now[qubits[i][0]] + r_now[qubits[i][0]]
            phase += r_now[qubits[i][0]] * (r_now[qubits[i][0]]-1)/2
        r_now = [r_next[i] for i in range(n_qudits)]
        s_now = [s_next[i] for i in range(n_qudits)]
    final = TensorProduct(X**(modulo_2(r_now[0])) * Z**(modulo_2(s_now[0])), X**(modulo_2(r_now[1])) * Z**(modulo_2(s_now[1])))
    for i in range(2, n_qudits):
        final = TensorProduct(final, X**(modulo_2(r_now[i])) * Z**(modulo_2(s_now[i])))
    
    display(omega**modulo_2(reduce_exponents(modulo_2(sym.simplify(phase)))) * final)
    return()



In [ ]:
def prepare_sym_candidates(P1,pi,pj):
    # prepare anti-commuting pauli strings with the same absolute coefficients for test of hadamard Symmetry
    C = Circuit(dimensions=[2 for i in range(P1.n_qudits())])
    # prime pauli pi and pj for cancel_pauli
    if P1.x_exp[pi, 0] == 1 and P1.z_exp[pj, 0] == 1: # x,z
        px = pi
        pz = pj
    elif P1.z_exp[pi, 0] == 1 and P1.x_exp[pj, 0] == 1: # z,x
        px = pj
        pz = pi
    elif P1.x_exp[pi, 0] == 1 and P1.z_exp[pj, 0] == 0: # x,id or x,x
        if any(P1.z_exp[pj, i] for i in range(P1.n_qudits())):
            g = CX(0,min([i for i in range(P1.n_qudits()) if P1.z_exp[pj, i]]),2)
        elif any(P1.x_exp[pj, i] for i in range(P1.n_qudits())):
            g = H(min([i for i in range(P1.n_qudits()) if P1.x_exp[pj, i]]),2)
            P1 = g.act(P1)
            C.add_gate(g)
            g = CX(0,min([i for i in range(P1.n_qudits()) if P1.z_exp[pj, i]]),2)
        C.add_gate(g)
        P1 = g.act(P1)
        px = pi
        pz = pj
    elif P1.z_exp[pi, 0] == 1 and P1.x_exp[pj, 0] == 0: # z,id or z,z
        if any(P1.x_exp[pj, i] for i in range(P1.n_qudits())):
            g = CX(min([i for i in range(P1.n_qudits()) if P1.x_exp[pj, i]]),0,2)
        elif any(P1.z_exp[pj, i] for i in range(P1.n_qudits())):
            g = H(min([i for i in range(P1.n_qudits()) if P1.z_exp[pj, i]]),2)
            P1 = g.act(P1)
            C.add_gate(g)
            g = CX(min([i for i in range(P1.n_qudits()) if P1.x_exp[pj, i]]),0,2)
        C.add_gate(g)
        P1 = g.act(P1)
        px = pj
        pz = pi
    elif P1.x_exp[pi, 0] == 0 and P1.z_exp[pj, 0] == 1: # id,z
        if any(P1.x_exp[pi, i] for i in range(P1.n_qudits())):
            g = CX(min([i for i in range(P1.n_qudits()) if P1.x_exp[pi, i]]),0,2)
        elif any(P.z_exp[pi, i] for i in range(P1.n_qudits())):
            g = H(min([i for i in range(P1.n_qudits()) if P1.z_exp[pi, i]]),2)
            P1 = g.act(P1)
            C.add_gate(g)
            g = CX(min([i for i in range(P1.n_qudits()) if P1.x_exp[pi, i]]),0,2)
        C.add_gate(g)
        P1 = g.act(P1)
        px = pi
        pz = pj
    elif P1.x_exp[pi, 0] == 0 and P1.x_exp[pj, 0] == 1: # id,x
        if any(P1.z_exp[pi, i] for i in range(P1.n_qudits())):
            g = CX(0,min([i for i in range(P1.n_qudits()) if P1.z_exp[pi, i]]),2)
        elif any(P1.x_exp[pi, i] for i in range(P1.n_qudits())):
            g = H(min([i for i in range(P1.n_qudits()) if P1.x_exp[pi, i]]),2)
            P1 = g.act(P1)
            C.add_gate(g)
            g = CX(0,min([i for i in range(P1.n_qudits()) if P1.z_exp[pi, i]]),2)
        C.add_gate(g)
        P1 = g.act(P1)
        px = pj
        pz = pi
    else: # id,id
        if any(P1.x_exp[pi, i] for i in range(P1.n_qudits())):
            g = CX(min([i for i in range(P1.n_qudits()) if P1.x_exp[pi, i]]),0,2)
            P1 = g.act(P1)
            C.add_gate(g)
            if any(P1.z_exp[pj, i] for i in range(P1.n_qudits())):
                g = CX(0,min([i for i in range(P1.n_qudits()) if P1.z_exp[pj, i]]),2)
            elif any(P1.x_exp[pj, i] for i in range(P1.n_qudits())):
                g = H(min([i for i in range(P1.n_qudits()) if P1.x_exp[pj, i]]),2)
                P1 = g.act(P1)
                C.add_gate(g)
                g = CX(0,min([i for i in range(P1.n_qudits()) if P1.z_exp[pj, i]]),2)
            C.add_gate(g)
            P1 = g.act(P1)
            px = pi
            pz = pj
        elif any(P1.z_exp[pi, i] for i in range(P1.n_qudits())):
            g = CX(0,min([i for i in range(P1.n_qudits()) if P1.z_exp[pi, i]]),2)
            P1 = g.act(P1)
            C.add_gate(g)
            if any(P1.x_exp[pj, i] for i in range(P1.n_qudits())):
                g = CX(min([i for i in range(P1.n_qudits()) if P1.x_exp[pj, i]]),0,2)
            elif any(P1.z_exp[pj, i] for i in range(P1.n_qudits())):
                g = H(min([i for i in range(P1.n_qudits()) if P1.z_exp[pj, i]]),2)
                P1 = g.act(P1)
                C.add_gate(g)
                g = CX(min([i for i in range(P1.n_qudits()) if P1.x_exp[pj, i]]),0,2)
            C.add_gate(g)
            P1 = g.act(P1)
            px = pj
            pz = pi
    return(C, P1, px, pz)

def check_current_paulis(P1,pi,pj):
    current_qubit_pauli_check = True
    for i in range(P1.n_qudits()):
        if i != pi and i != pj:
            if P1.x_exp[i, 0] == 1 and P1.z_exp[i, 0] == 1: # Y
                continue
            elif P1.x_exp[i, 0] == 0 and P1.z_exp[i, 0] == 0: # I
                continue
            elif (P1.x_exp[i, 0] == 1 and P1.z_exp[i, 0] == 0) or (P1.x_exp[i, 0] == 0 and P1.z_exp[i, 0] == 1): # X, Z
                if i not in flatten_list(cc_bands):
                    current_qubit_pauli_check = False
                    break
                else:
                    for b2 in cc_bands:
                        if i in b2:
                            if sum([P1.x_exp[j, 0] for j in b2]) - sum([P1.z_exp[j, 0] for j in b2]) == 0:
                                continue
                            else:
                                current_qubit_pauli_check = False
                                break
    return(current_qubit_pauli_check)

def remove_Ys(P1):
    sym_paulis_candidates = [i for i in range(P1.n_paulis()) if (P1.x_exp[i,0] == 1 and P1.z_exp[i,0] == 0) or (P1.x_exp[i,0] == 0 and P1.z_exp[i,0] == 1)]
    #print('sym_paulis_candidates',sym_paulis_candidates)
    other_paulis = [i for i in range(P1.n_paulis()) if i not in sym_paulis_candidates]
    #print('other_paulis',other_paulis)
    P2 = P1.copy()
    h_sym = False
    h_sym_impossible = False
    no_forcings = False
    no_qubits = False
    usable_qubits = [i for i in range(1,P1.n_qudits())]
    gate_options = [[0,1,2,3] for i in range(P1.n_qudits()-1)]
    #print(gate_options)
    while not no_forcings and len(usable_qubits) > 0 and not h_sym and not h_sym_impossible:
        list_n_I = [number_of_I(P2,Pauli_index,usable_qubits) for Pauli_index in range(P2.n_paulis())]
        #print('list_n_I',list_n_I)
        forcing_candidates_paulis = [i for i in range(P2.n_paulis()) if list_n_I[i] == len(usable_qubits) - 1 and i not in sym_paulis_candidates]
        #print('forcing_candidates_paulis',forcing_candidates_paulis)
        forcing_candidates_qubits = []
        for i in forcing_candidates_paulis:
            qubit_index = min([j for j in usable_qubits if (P2.x_exp[i,j]+P2.z_exp[i,j]) != 0])
            if qubit_index not in forcing_candidates_qubits:
                forcing_candidates_qubits.append(qubit_index)
        #print('forcing_candidates_qubits',forcing_candidates_qubits)
        if len(forcing_candidates_qubits) == 0:
            no_forcings = True

        no_forcings = True
        for qi in forcing_candidates_qubits:
            pauli_indexes = [i for i in forcing_candidates_paulis if (P2.x_exp[i,qi]+P2.z_exp[i,qi]) != 0]
            #print('pauli_indexes',pauli_indexes)
            first_qubit_x = P2.x_exp[[j for j in pauli_indexes],0]
            #print('first_qubit_x',first_qubit_x)
            if len(pauli_indexes) > 1: 
                if not any(first_qubit_x):
                    #print(gate_options[qi-1])
                    gate_options[qi-1].remove(1)
                    gate_options[qi-1].remove(2)
                    gate_options[qi-1].remove(3)
                    usable_qubits.remove(qi)
                    # identity
                elif not any((first_qubit_x + P2.x_exp[[j for j in pauli_indexes],qi])%2):
                    gate_options[qi-1].remove(0)
                    gate_options[qi-1].remove(2)
                    gate_options[qi-1].remove(3)
                    g = add_r2(qi)
                    P2 = g.act(P2)
                    usable_qubits.remove(qi)
                elif not any((first_qubit_x + P2.z_exp[[j for j in pauli_indexes],qi])%2):
                    gate_options[qi-1].remove(0)
                    gate_options[qi-1].remove(1)
                    gate_options[qi-1].remove(3)
                    g = add_s2(qi)
                    P2 = g.act(P2)
                    usable_qubits.remove(qi)
                elif not any((first_qubit_x + P2.z_exp[[j for j in pauli_indexes],qi] + P2.x_exp[[j for j in pauli_indexes],qi])%2):
                    #print(gate_options[qi-1])
                    gate_options[qi-1].remove(0)
                    gate_options[qi-1].remove(1)
                    gate_options[qi-1].remove(2)
                    g = add_r2s2(qi)
                    P2 = g.act(P2)
                    usable_qubits.remove(qi)
                else:
                    gate_options[qi-1].remove(0)
                    gate_options[qi-1].remove(1)
                    gate_options[qi-1].remove(2)
                    gate_options[qi-1].remove(3)
                    h_sym_impossible = True
                    #print('Not Possible')
                    usable_qubits.remove(qi)
                no_forcings = False
                break
            else:
                continue
        first_qubit_x = P2.x_exp[other_paulis,0]
        #print('usable_qubits',usable_qubits)
        #print('gate_options',gate_options)
        if not any(first_qubit_x%2):
            h_sym = True
    return(P2,gate_options,h_sym,h_sym_impossible)

In [36]:
def Find_Hadamard_Symmetries(P):
    # Step 0
    P1 = P.copy()
    #print(P1)
    cc = P1.weights
    # Step 1, organize the coefficients into bands with the same absolute value
    cc_abs = np.abs(cc)
    cc_bands = group_indices(cc_abs)
    cc_bands = [np.array(b) for b in cc_bands if len(b) > 1]

    print(cc_bands)
    print([cc[b[0]] for b in cc_bands])

    for ib,b in enumerate(cc_bands):
        for ic,pi in enumerate(b):
            for jc,pj in enumerate(b[ic+1:]):
                P1 = P.copy()
                if not P1.is_commuting(pauli_string_indexes=(pi,pj)):
                    print(f"Band {ib}, Pauli {pi} and {pj} anti-commute")
                    #print()
                    C, P1, px, pz = prepare_sym_candidates(P1,pi,pj)

                    #print('Prepared:',pi,pj)
                    #print(P1)
                    #print()
                    # cancel for pauli with x
                    P1, C = cancel_pauli(P1, 0, px, C, P1.n_qudits())
                    # cancel for pauli with z
                    g = H(0, P.dimensions[0])
                    C.add_gate(g)
                    P1 = g.act(P1)
                    P1, C = cancel_pauli(P1, 0, pz, C, P1.n_qudits())  
                    g = H(0, P.dimensions[0])
                    C.add_gate(g)
                    P1 = g.act(P1)
                    #print('Make X111... and Z111...')
                    #print(P1)
                    #print()

                    # check if all other paulis in the first qubit have either id, Y, or x and z but with the same coefficients
                    current_qubit_pauli_check = check_current_paulis(P1,pi,pj)
                    #print('current_qubit_pauli_check',current_qubit_pauli_check)
                    if not current_qubit_pauli_check:
                        continue
                    
                    #print(pi,pj,'are candidates for Hadamard symmetry')
                    
                    # organize the remaining paulis to make detection of useful qubits easier
                    for i in range(1,P1.n_qudits()):
                        C, pivots = symplectic_reduction_iter_qudit_(P.copy(), C, [], i)
                    P1 = C.act(P.copy())
                    #print(P1)
                    #print()

                    # cancel the Y's in the first qubit
                    sym_paulis_candidates = [i for i in range(P1.n_paulis()) if (P1.x_exp[i,0] == 1 and P1.z_exp[i,0] == 0) or (P1.x_exp[i,0] == 0 and P1.z_exp[i,0] == 1)]
                    #print('sym_paulis_candidates',sym_paulis_candidates)
                    other_paulis = [i for i in range(P1.n_paulis()) if i not in sym_paulis_candidates]
                    #print('other_paulis',other_paulis)
                    P2 = P1.copy()
                    h_sym = False
                    h_sym_impossible = False
                    no_forcings = False
                    no_qubits = False
                    usable_qubits = [i for i in range(1,P1.n_qudits())]
                    gate_options = [[0,1,2,3] for i in range(P1.n_qudits()-1)]
                    #print(gate_options)
                    while not no_forcings and len(usable_qubits) > 0 and not h_sym and not h_sym_impossible:
                        list_n_I = [number_of_I(P2,Pauli_index,usable_qubits) for Pauli_index in range(P.n_paulis())]
                        #print('list_n_I',list_n_I)
                        forcing_candidates_paulis = [i for i in range(P.n_paulis()) if list_n_I[i] == len(usable_qubits) - 1 and i not in sym_paulis_candidates]
                        #print('forcing_candidates_paulis',forcing_candidates_paulis)
                        forcing_candidates_qubits = []
                        for i in forcing_candidates_paulis:
                            qubit_index = min([j for j in usable_qubits if (P1.x_exp[i,j]+P1.z_exp[i,j]) != 0])
                            if qubit_index not in forcing_candidates_qubits:
                                forcing_candidates_qubits.append(qubit_index)
                        #print('forcing_candidates_qubits',forcing_candidates_qubits)
                        if len(forcing_candidates_qubits) == 0:
                            no_forcings = True

                        no_forcings = True
                        for qi in forcing_candidates_qubits:
                            pauli_indexes = [i for i in forcing_candidates_paulis if (P2.x_exp[i,qi]+P2.z_exp[i,qi]) != 0]
                            #print('pauli_indexes',pauli_indexes)
                            first_qubit_x = P2.x_exp[[j for j in pauli_indexes],0]
                            #print('first_qubit_x',first_qubit_x)
                            if len(pauli_indexes) > 1: 
                                if not any(first_qubit_x):
                                    #print(gate_options[qi-1])
                                    gate_options[qi-1].remove(1)
                                    gate_options[qi-1].remove(2)
                                    gate_options[qi-1].remove(3)
                                    usable_qubits.remove(qi)
                                    # identity
                                elif not any((first_qubit_x + P1.x_exp[[j for j in pauli_indexes],qi])%2):
                                    gate_options[qi-1].remove(0)
                                    gate_options[qi-1].remove(2)
                                    gate_options[qi-1].remove(3)
                                    g = add_r2(qi)
                                    P2 = g.act(P2)
                                    usable_qubits.remove(qi)
                                elif not any((first_qubit_x + P1.z_exp[[j for j in pauli_indexes],qi])%2):
                                    gate_options[qi-1].remove(0)
                                    gate_options[qi-1].remove(1)
                                    gate_options[qi-1].remove(3)
                                    g = add_s2(qi)
                                    P2 = g.act(P2)
                                    usable_qubits.remove(qi)
                                elif not any((first_qubit_x + P1.z_exp[[j for j in pauli_indexes],qi] + P1.x_exp[[j for j in pauli_indexes],qi])%2):
                                    #print(gate_options[qi-1])
                                    gate_options[qi-1].remove(0)
                                    gate_options[qi-1].remove(1)
                                    gate_options[qi-1].remove(2)
                                    g = add_r2s2(qi)
                                    P2 = g.act(P2)
                                    usable_qubits.remove(qi)
                                else:
                                    gate_options[qi-1].remove(0)
                                    gate_options[qi-1].remove(1)
                                    gate_options[qi-1].remove(2)
                                    gate_options[qi-1].remove(3)
                                    h_sym_impossible = True
                                    #print('Not Possible')
                                    usable_qubits.remove(qi)
                                no_forcings = False
                                break
                            else:
                                continue
                        first_qubit_x = P2.x_exp[other_paulis,0]
                        #print('usable_qubits',usable_qubits)
                        #print('gate_options',gate_options)
                        if not any(first_qubit_x%2):
                            h_sym = True

                    #print(P2)
                    if not h_sym and not h_sym_impossible:
                        addition_options_lens = []
                        for l in gate_options:
                            addition_options_lens.append(len(l))
                        max_options = np.prod(addition_options_lens)
                        for i in range(max_options):
                            addition_options = int_to_bases(i,dims=addition_options_lens)
                            first_qubit_x = P2.x_exp[[j for j in range(P1.n_paulis()) if j not in sym_paulis_candidates],0]
                            for k in range(len(addition_options)):
                                if addition_options[k] == 0:
                                    pass
                                elif addition_options[k] == 1:
                                    first_qubit_x += P2.x_exp[other_paulis,k+1]
                                elif addition_options[k] == 2:
                                    first_qubit_x += P2.z_exp[other_paulis,k+1]
                                elif addition_options[k] == 3:
                                    first_qubit_x += P2.x_exp[other_paulis,k+1] + P2.z_exp[other_paulis,k+1]
                            #print(first_qubit_x%2)
                            if not any(first_qubit_x%2):
                                #print('found solution')
                                #print(addition_options)
                                for j,k in enumerate(addition_options):
                                    if addition_options_lens[j] > 1:
                                        if k == 0:
                                            #print(j,k)
                                            pass
                                        elif k == 1:
                                            #print(j,k)
                                            g = add_r2(j+1)
                                            P2 = g.act(P2)
                                        elif k == 2:
                                            #print(j,k)
                                            g = add_s2(j+1)
                                            P2 = g.act(P2)
                                        elif k == 3:
                                            #print(j,k)
                                            g = add_r2s2(j+1)
                                            P2 = g.act(P2) 
                                break
                    #print()
                    #print(P2)
                    first_qubit_x = P2.x_exp[other_paulis,0]
                    #print(first_qubit_x%2)
                    if any(first_qubit_x%2):
                        print('No Hadamard Symmetry')
                    else:
                        return(P2)
                        # Erase phase
                        #               

In [6]:
P,C = Hadamard_Symmetric_PauliSum(12,8,3)
print(P)

[0.2209005]
[0.97127789]
[1.15148576]
[10, 2, 7] [9, 1, 6]
(-0.059091729231770806+0j)|x0z1 x1z1 x0z0 x1z1 x1z1 x0z1 x0z0 x0z1 | 0 
(0.9712778895141534+0j)   |x1z1 x0z0 x0z1 x1z1 x1z1 x1z0 x0z1 x0z1 | 0 
(-0.9712778895141534+0j)  |x1z1 x0z0 x1z0 x1z1 x1z0 x1z1 x1z0 x1z0 | 0 
(0.30234662208083557+0j)  |x0z0 x0z1 x1z0 x0z0 x1z0 x1z1 x0z0 x1z0 | 0 
(-0.606026972535971+0j)   |x1z0 x0z1 x0z0 x0z0 x0z1 x1z0 x0z0 x0z1 | 0 
(0.35490400729218674+0j)  |x1z1 x0z0 x1z0 x0z0 x1z1 x1z0 x0z0 x0z1 | 0 
(-1.151485756874005+0j)   |x0z0 x1z1 x1z1 x0z1 x0z1 x1z0 x0z1 x1z1 | 0 
(-1.151485756874005+0j)   |x0z1 x0z1 x1z1 x0z0 x1z0 x0z0 x1z0 x0z0 | 0 
(-1.7856508070838983+0j)  |x1z1 x0z0 x0z0 x1z1 x1z1 x1z0 x1z1 x0z0 | 0 
(-0.22090050415433887+0j) |x1z1 x0z0 x1z1 x1z0 x1z0 x0z0 x1z1 x0z1 | 0 
(-0.22090050415433887+0j) |x1z0 x0z1 x1z1 x1z1 x0z0 x1z0 x0z1 x0z0 | 0 
(0.11228359566793164+0j)  |x1z0 x0z1 x1z1 x1z1 x1z1 x0z0 x0z0 x1z0 | 0 



In [37]:
print(Find_Hadamard_Symmetries(P))

[array([1, 2]), array([6, 7]), array([ 9, 10])]
[(0.9712778895141534+0j), (-1.151485756874005+0j), (-0.22090050415433887+0j)]
Band 0, Pauli 1 and 2 anti-commute
(-0.059091729231770806+0j)|x0z0 x1z0 x0z0 x0z0 x0z0 x0z0 x0z0 x0z0 | 0 
(0.9712778895141534+0j)   |x1z0 x0z0 x0z0 x1z0 x1z0 x0z0 x0z1 x0z0 | 0 
(-0.9712778895141534+0j)  |x0z1 x0z0 x0z0 x1z0 x1z0 x0z0 x0z1 x0z0 | 0 
(0.30234662208083557+0j)  |x0z0 x0z0 x1z0 x0z0 x0z0 x0z0 x0z0 x0z0 | 1 
(-0.606026972535971+0j)   |x0z0 x0z0 x0z1 x0z0 x0z0 x0z0 x0z0 x0z0 | 0 
(0.35490400729218674+0j)  |x0z0 x0z0 x0z1 x0z1 x0z0 x0z0 x0z0 x0z0 | 0 
(-1.151485756874005+0j)   |x0z0 x0z0 x0z1 x1z1 x1z0 x0z0 x1z0 x0z0 | 1 
(-1.151485756874005+0j)   |x0z0 x0z0 x0z1 x1z0 x0z0 x0z0 x0z0 x0z0 | 1 
(-1.7856508070838983+0j)  |x0z0 x0z0 x0z0 x0z1 x0z0 x1z0 x0z0 x0z0 | 1 
(-0.22090050415433887+0j) |x0z0 x0z1 x0z0 x0z0 x0z0 x0z0 x0z0 x0z0 | 0 
(-0.22090050415433887+0j) |x0z0 x1z1 x0z0 x0z0 x1z0 x0z0 x0z0 x0z0 | 0 
(0.11228359566793164+0j)  |x0z0 x0z1 x0z0 x1z0 